In [1]:
import pandas as pd

In [2]:
matches = pd.read_csv('matches.csv', index_col=0)

In [3]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,14.0,2.0,18.4,0.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,1.0,0.0,17.0,0.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,1.0,0.0,7.2,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,6.0,1.0,10.8,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,10.0,2.0,19.4,0.0,0.0,0.0,2022,Manchester City


In [4]:
matches.shape

(1494, 27)

There are 38 matches and 20 teams each season in an EPL game; We have data for 2 seasons 2021 & 2022.  So the data should have 38x20x2 rows.

In [5]:
38 * 20 * 2

1520

In [6]:
matches.team.value_counts()

Manchester City             75
Wolverhampton Wanderers     75
Leeds United                75
Southampton                 75
Liverpool                   75
Brighton and Hove Albion    75
Newcastle United            75
West Ham United             75
Manchester United           75
Arsenal                     75
Tottenham Hotspur           75
Leicester City              74
Crystal Palace              74
Aston Villa                 74
Everton                     74
Chelsea                     74
Burnley                     74
Fulham                      38
West Bromwich Albion        38
Sheffield United            38
Brentford                   37
Watford                     37
Norwich City                37
Name: team, dtype: int64

In [11]:
matches['round'].value_counts()

Matchweek 1     40
Matchweek 19    40
Matchweek 21    40
Matchweek 22    40
Matchweek 23    40
Matchweek 24    40
Matchweek 25    40
Matchweek 26    40
Matchweek 28    40
Matchweek 29    40
Matchweek 31    40
Matchweek 32    40
Matchweek 30    40
Matchweek 34    40
Matchweek 35    40
Matchweek 36    40
Matchweek 37    40
Matchweek 2     40
Matchweek 20    40
Matchweek 10    40
Matchweek 16    40
Matchweek 15    40
Matchweek 14    40
Matchweek 13    40
Matchweek 12    40
Matchweek 3     40
Matchweek 11    40
Matchweek 17    40
Matchweek 9     40
Matchweek 8     40
Matchweek 7     40
Matchweek 6     40
Matchweek 5     40
Matchweek 4     40
Matchweek 33    38
Matchweek 18    38
Matchweek 27    38
Matchweek 38    20
Name: round, dtype: int64

In [12]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk              float64
pkatt           float64
season            int64
team             object
dtype: object

In [13]:
matches['date'] = pd.to_datetime(matches['date'])

In [15]:
matches['venue_code'] = matches['venue'].astype('category').cat.codes

In [17]:
matches['opp_code'] = matches['opponent'].astype('category').cat.codes

In [19]:
matches['hour'] = matches['time'].str.replace(':.+', '', regex=True).astype('int')

In [22]:
matches['day_code'] = matches['date'].dt.dayofweek

In [25]:
matches['target'] = (matches['result'] == 'W').astype('int')

In [26]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,0.0,0.0,0.0,2022,Manchester City,0,18,16,6,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,0.0,0.0,0.0,2022,Manchester City,1,15,15,5,1
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,0.0,0.0,0.0,2022,Manchester City,1,0,12,5,1
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,0.0,0.0,0.0,2022,Manchester City,0,10,15,5,1
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,0.0,0.0,0.0,2022,Manchester City,1,17,15,5,0


In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [29]:
train = matches[matches['date'] < '2022-01-01']

In [30]:
test = matches[matches['date'] > '2022-01-01']

In [31]:
predictors = ['venue_code', 'opp_code', 'hour', 'day_code']

In [32]:
rf.fit(train[predictors], train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [33]:
pred = rf.predict(test[predictors])

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
acc = accuracy_score(test['target'], pred)

In [36]:
acc

0.5994475138121547

In [37]:
combined = pd.DataFrame(dict(actual=test['target'], predictions=pred))

In [38]:
pd.crosstab(index=combined['actual'], columns=combined['predictions'])

predictions,0,1
actual,,
0,175,43
1,102,42


In [39]:
from sklearn.metrics import precision_score

In [40]:
precision_score(test['target'], pred)

0.49411764705882355

Improving accuracy of the model

In [41]:
grouped_matches = matches.groupby('team')

In [42]:
group = grouped_matches.get_group('Manchester City')

In [43]:
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,0.0,0.0,0.0,2022,Manchester City,0,18,16,6,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,0.0,0.0,0.0,2022,Manchester City,1,15,15,5,1
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,0.0,0.0,0.0,2022,Manchester City,1,0,12,5,1
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,0.0,0.0,0.0,2022,Manchester City,0,10,15,5,1
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,0.0,0.0,0.0,2022,Manchester City,1,17,15,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,2021-05-01,12:30,Premier League,Matchweek 34,Sat,Away,W,2.0,0.0,Crystal Palace,...,0.0,0.0,0.0,2021,Manchester City,0,6,12,5,1
56,2021-05-08,17:30,Premier League,Matchweek 35,Sat,Home,L,1.0,2.0,Chelsea,...,1.0,0.0,0.0,2021,Manchester City,1,5,17,5,0
57,2021-05-14,20:00,Premier League,Matchweek 36,Fri,Away,W,4.0,3.0,Newcastle Utd,...,1.0,1.0,2.0,2021,Manchester City,0,14,20,4,1
58,2021-05-18,19:00,Premier League,Matchweek 37,Tue,Away,L,2.0,3.0,Brighton,...,1.0,0.0,0.0,2021,Manchester City,0,3,19,1,0


In [44]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    group[new_cols] = group[cols].rolling(3, closed='left').mean()
    group = group.dropna(subset=new_cols)
    return group

In [45]:
cols = ['gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt']
new_cols = [f'{c}_rolling' for c in cols]

In [46]:
new_cols

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [47]:
matches_rolling = matches.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))

In [49]:
matches_rolling

date   time            comp         round  \
team                                                                         
Arsenal                 6  2020-10-04  14:00  Premier League   Matchweek 4   
                        7  2020-10-17  17:30  Premier League   Matchweek 5   
                        9  2020-10-25  19:15  Premier League   Matchweek 6   
                        11 2020-11-01  16:30  Premier League   Matchweek 7   
                        13 2020-11-08  19:15  Premier League   Matchweek 8   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 36 2022-04-24  14:00  Premier League  Matchweek 34   
                        37 2022-04-30  15:00  Premier League  Matchweek 35   
                        38 2022-05-07  15:00  Premier League  Matchweek 36   
                        39 2022-05-11  20:15  Premier League  Matchweek 33   
                        40 2022-05-15  14:00  Premier League  Matchweek 37   

                            day venue result   gf   ga         opponent  ...  \
team                                                                     ...   
Arsenal                 6   Sun  Home      W  2.0  1.0    Sheffield Utd  ...   
                        7   Sat  Away      L  0.0  1.0  Manchester City  ...   
                        9   Sun  Home      L  0.0  1.0   Leicester City  ...   
                        11  Sun  Away      W  1.0  0.0   Manchester Utd  ...   
                        13  Sun  Home      L  0.0  3.0      Aston Villa  ...   
...                         ...   ...    ...  ...  ...              ...  ...   
Wolverhampton Wanderers 36  Sun  Away      L  0.0  1.0          Burnley  ...   
                        37  Sat  Home      L  0.0  3.0         Brighton  ...   
                        38  Sat  Away      D  2.0  2.0          Chelsea  ...   
                        39  Wed  Home      L  1.0  5.0  Manchester City  ...   
                        40  Sun  Home      D  1.0  1.0     Norwich City  ...   

                            day_code  target  gf_rolling  ga_rolling  \
team                                                                   
Arsenal                 6          6       1    2.000000    1.333333   
                        7          5       0    1.666667    1.666667   
                        9          6       0    1.000000    1.666667   
                        11         6       1    0.666667    1.000000   
                        13         6       0    0.333333    0.666667   
...                              ...     ...         ...         ...   
Wolverhampton Wanderers 36         6       0    1.333333    1.666667   
                        37         5       0    0.666667    1.000000   
                        38         5       0    0.000000    1.666667   
                        39         2       0    0.666667    2.000000   
                        40         6       0    1.000000    3.333333   

                           sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                        
Arsenal                 6   13.666667    4.666667    20.066667   0.000000   
                        7   14.000000    4.666667    19.433333   0.000000   
                        9   13.333333    5.333333    19.233333   0.000000   
                        11   8.333333    3.333333    23.700000   0.333333   
                        13   9.333333    3.333333    21.833333   0.333333   
...                               ...         ...          ...        ...   
Wolverhampton Wanderers 36  13.333333    4.000000    17.300000   1.000000   
                        37  12.666667    4.000000    19.133333   0.666667   
                        38  13.000000    5.333333    19.533333   0.333333   
                        39  15.666667    6.000000    18.800000   0.666667   
                        40  16.333333    6.000000    17.166667   0.666667   

                            pk_rolling

In [50]:
matches_rolling = matches_rolling.droplevel('team')

In [52]:
matches_rolling.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,...,6,1,2.000000,1.333333,13.666667,4.666667,20.066667,0.000000,0.0,0.0
7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,...,5,0,1.666667,1.666667,14.000000,4.666667,19.433333,0.000000,0.0,0.0
9,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,...,6,0,1.000000,1.666667,13.333333,5.333333,19.233333,0.000000,0.0,0.0
11,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,...,6,1,0.666667,1.000000,8.333333,3.333333,23.700000,0.333333,0.0,0.0
13,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,...,6,0,0.333333,0.666667,9.333333,3.333333,21.833333,0.333333,0.0,0.0


In [53]:
matches_rolling.index = range(matches_rolling.shape[0])

In [54]:
matches_rolling.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,...,6,1,2.000000,1.333333,13.666667,4.666667,20.066667,0.000000,0.0,0.0
1,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,...,5,0,1.666667,1.666667,14.000000,4.666667,19.433333,0.000000,0.0,0.0
2,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,...,6,0,1.000000,1.666667,13.333333,5.333333,19.233333,0.000000,0.0,0.0
3,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,...,6,1,0.666667,1.000000,8.333333,3.333333,23.700000,0.333333,0.0,0.0
4,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,...,6,0,0.333333,0.666667,9.333333,3.333333,21.833333,0.333333,0.0,0.0


In [56]:
def make_prediction(data, predictors):
    train = data[data['date'] < '2022-01-01']
    test = data[data['date'] > '2022-01-01']
    rf.fit(train[predictors], train['target'])
    pred = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test['target'], predictions=pred), index=test.index)
    precision = precision_score(test['target'], pred)
    return combined, precision

In [57]:
combined, precision = make_prediction(matches_rolling, predictors + new_cols)

In [58]:
precision

0.5540540540540541

In [59]:
combined

,actual,predictions
55,0,1
56,1,1
57,1,0
58,1,0
59,1,0
...,...,...
1417,0,0
1418,0,0
1419,0,0
1420,0,0


In [60]:
combined = combined.merge(matches_rolling[['date', 'team', 'opponent', 'result']], left_index=True, right_index=True)

In [61]:
combined

,actual,predictions,date,team,opponent,result
55,0,1,2022-01-23,Arsenal,Burnley,D
56,1,1,2022-02-10,Arsenal,Wolves,W
57,1,0,2022-02-19,Arsenal,Brentford,W
58,1,0,2022-02-24,Arsenal,Wolves,W
59,1,0,2022-03-06,Arsenal,Watford,W
...,...,...,...,...,...,...
1417,0,0,2022-04-24,Wolverhampton Wanderers,Burnley,L
1418,0,0,2022-04-30,Wolverhampton Wanderers,Brighton,L
1419,0,0,2022-05-07,Wolverhampton Wanderers,Chelsea,D
1420,0,0,2022-05-11,Wolverhampton Wanderers,Manchester City,L


In [64]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

In [65]:
map_values = {
    'Brighton and Hove Albion': 'Brighton',
    'Manchester United': 'Manchester Utd',
    'Newcastle United': 'Newcastle Utd',
    'Tottenham Hotspur': 'Tottenham',
    'West Ham United': 'West Ham',
    'Wolverhampton Wanderers': 'Wolves'
}

In [66]:
mapping = MissingDict(**map_values)

In [68]:
mapping['Brighton and Hove Albion']

'Brighton'

In [69]:
combined['new_team'] = combined['team'].map(mapping)

In [70]:
combined

,actual,predictions,date,team,opponent,result,new_team
55,0,1,2022-01-23,Arsenal,Burnley,D,Arsenal
56,1,1,2022-02-10,Arsenal,Wolves,W,Arsenal
57,1,0,2022-02-19,Arsenal,Brentford,W,Arsenal
58,1,0,2022-02-24,Arsenal,Wolves,W,Arsenal
59,1,0,2022-03-06,Arsenal,Watford,W,Arsenal
...,...,...,...,...,...,...,...
1417,0,0,2022-04-24,Wolverhampton Wanderers,Burnley,L,Wolves
1418,0,0,2022-04-30,Wolverhampton Wanderers,Brighton,L,Wolves
1419,0,0,2022-05-07,Wolverhampton Wanderers,Chelsea,D,Wolves
1420,0,0,2022-05-11,Wolverhampton Wanderers,Manchester City,L,Wolves


In [71]:
marged = combined.merge(combined, left_on=['date', 'new_team'], right_on=['date', 'opponent'])

In [72]:
marged

,actual_x,predictions_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predictions_y,team_y,opponent_y,result_y,new_team_y
0,0,1,2022-01-23,Arsenal,Burnley,D,Arsenal,0,0,Burnley,Arsenal,D,Burnley
1,1,1,2022-02-10,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
2,1,0,2022-02-19,Arsenal,Brentford,W,Arsenal,0,0,Brentford,Arsenal,L,Brentford
3,1,0,2022-02-24,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
4,1,0,2022-03-06,Arsenal,Watford,W,Arsenal,0,0,Watford,Arsenal,L,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,0,0,2022-04-24,Wolverhampton Wanderers,Burnley,L,Wolves,1,0,Burnley,Wolves,W,Burnley
358,0,0,2022-04-30,Wolverhampton Wanderers,Brighton,L,Wolves,1,0,Brighton and Hove Albion,Wolves,W,Brighton
359,0,0,2022-05-07,Wolverhampton Wanderers,Chelsea,D,Wolves,0,1,Chelsea,Wolves,D,Chelsea
360,0,0,2022-05-11,Wolverhampton Wanderers,Manchester City,L,Wolves,1,1,Manchester City,Wolves,W,Manchester City


In [73]:
marged[(marged['predictions_x'] == 1) & (marged['predictions_y'] == 0)]['actual_x'].value_counts()

1    39
0    31
Name: actual_x, dtype: int64

In [74]:
39 / 70

0.5571428571428572